# Notebook para leer los txt

Se encarga de pasar los datos de los archivos a un dataframe con la información sobre el paciente, número de repetición, el id del movimiento, si es correcto y la posición de los keypoints. Después, se crea otro dataframe con los ángulos entre algunos de los keypoits.

## 1. Raw Data Folder

* 30 participantes (14 sanos y 16 pacientes)
* 9 ejercicios
* x6 repeticiones
* 30 fps
* 2598 videos

In [2]:
# importar librerias necesarias
import pandas as pd # para manejar dataframes
import os # para interactur con el sistema operativo
import numpy as np

### 1.1 Datos de los nombres de los archivos

SubjectID_DateID_GestureLabel_RepetitionNumber_CorrectLabel_Position.txt

* SubjectID: id uniquely identifying the person performing the exercise
* DateID: id identifying the session in which the person was recorded
* GestureLabel: Label identifying the gesture; possible values are from 0 to 8
* RepetitionNumber: Each gesture was repeated several times and this shows the repetition number
* CorrectLabel: A value of 1 represents a gesture labeled as being correctly executed, while a value of 2 is for a gesture labeled as incorrect
* Position: Some of the persons performed the gestures sitting on a chair or wheelchair, while others standing

In [3]:
# función para guardar en el dataframe los datos que aparecen en los nombre de los archivos
def leer_nombre_archivo(archivo:str):
    """
    Obtiene los datos de interés del nombre de los archivos

    :param archivo: nombre del archivo (SubjectID_DateID_GestureLabel_RepetitionNumber_CorrectLabel_Position.txt)
    :return: devuelve una lista de strings con los datos del nombre del archivo
    (SubjectID, DateID, GestureLabel, RepetitionNumber, CorrectLabel, Position)
    """ 
    archivo = archivo.split('.')[0] # quita la extension txt
    campos = archivo.split('_') # separa los campos por _
    return campos

### 1.2 Datos de los archivos
Each raw data file contains per line: timestamp, XX, XX, followed by a 25 pairs of (JointName, TrackedStatus, 3d coordinate X, 3d coordinate Y, 3d coordinate Z, 2d coordinate X, 2d coordinate Y)

In [4]:
def leer_datos_archivo(directorio:str, columnas:list[str]):

    """
    Compila toda la información de los archivos de un directorio y los guarda en un dataframe

    :param directorio: Nombre del directirio donde están los archivos
    :param columnas: Lista con el nombre de las columnas que queremos que tenga el dataframe de salida
    :return: DataFrame con todos los datos recompilados
    """
    file_list = os.listdir(directorio) # crea una lista con los nombre de los arhivos que se encuentran en ese directorio

    # Initialize an empty list to store the extracted data
    list_data = []

    # Iterate through each raw data file
    for file_name in file_list:
        campos = leer_nombre_archivo(file_name)

        with open(os.path.join(directorio, file_name), 'r') as file:
            for line in file:
                # Split the line by comma and extract the required information
                line_data = line.strip().split(',')[3:]  # Skip the initial timestamp and other unwanted data
                cleaned_data = [item.replace('(', '').replace(')', '') for item in line_data] # quitar los paréntesis
                for i in range(0, len(cleaned_data), 7): # por cada linea de los archivos necesitamos bloques de 7 valores
                    list_data.append(campos + cleaned_data[i:i + 7])
    df = pd.DataFrame(list_data, columns=columnas)
    return df


In [5]:
directory = '../dataset/SkeletonData/RawData' # directorio donde se encuentran los datos

columnas = ['SubjectID', 'DateID', 'GestureLabel', 'RepetitionNumber', 'CorrectLabel', 'Position',
            'JointName', 'TrackedStatus', '3D_X', '3D_Y', '3D_Z', '2D_X', '2D_Y']

df_data = leer_datos_archivo(directory, columnas)


KeyboardInterrupt: 

In [ ]:
df_data

,SubjectID,DateID,GestureLabel,RepetitionNumber,CorrectLabel,Position,JointName,TrackedStatus,3D_X,3D_Y,3D_Z,2D_X,2D_Y
0,101,18,0,1,1,stand,SpineBase,Tracked,-0.1028086,0.06965441,2.464606,243.133,196.5854
1,101,18,0,1,1,stand,SpineMid,Tracked,-0.1026228,0.3837799,2.438919,242.971,149.1077
2,101,18,0,1,1,stand,Neck,Tracked,-0.1025293,0.6877351,2.40196,242.6967,101.3569
3,101,18,0,1,1,stand,Head,Tracked,-0.1190992,0.8358598,2.373549,239.9086,76.90204
4,101,18,0,1,1,stand,ShoulderLeft,Tracked,-0.2826451,0.5525576,2.38421,214.7496,121.544
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5707520,307,18,8,9,1,stand,SpineShoulder,Tracked,-0.05799517,0.5291457,2.422904,249.6275,126.5549
5707521,307,18,8,9,1,stand,HandTipLeft,Tracked,-0.302538,-0.1131345,2.284269,209.7731,225.1547
5707522,307,18,8,9,1,stand,ThumbLeft,Tracked,-0.2783904,-0.06298634,2.269769,213.3857,217.1493
5707523,307,18,8,9,1,stand,HandTipRight,Tracked,0.1525867,-0.136378,2.45287,281.2744,227.365


In [ ]:
df_data.to_csv('../csvFiles/raw_pacientes.csv', index=False) # lo guardamos a cvs

## 2. Calcular los ángulos

<div>
<img src="../images/gestures.png" width="500"/>
</div>

In [ ]:
def calculate_angle(df: pd.DataFrame, joint_a: str, joint_b: str, joint_c: str):
    """
    Calcula los ángulos usando tres keypoints como referencia
    """
    # Extract positions of keypoints
    positions = df.set_index('JointName')[['3D_X', '3D_Y']].loc[[joint_a, joint_b, joint_c]]

    # Convert positions to numeric
    positions = positions.apply(pd.to_numeric)

    # Calculate radians and angle
    radians = np.arctan2(positions.iloc[2, 1] - positions.iloc[1, 1], positions.iloc[2, 0] - positions.iloc[1, 0]) - np.arctan2(positions.iloc[0, 1] - positions.iloc[1, 1], positions.iloc[0, 0] - positions.iloc[1, 0])
    angle = np.abs(radians * 180.0 / np.pi)

    # Ensure angle is within 180 degrees
    if angle > 180.0:
        angle = 360 - angle

    return angle



In [ ]:
# Group the DataFrame by every 25 rows
groups = [df_data.iloc[i:i+25] for i in range(0, len(df_data), 25)]

# Calculate angles for each group
angles = []
for group in groups:
    elbow_angle_left = calculate_angle(group, 'ShoulderLeft', 'ElbowLeft', 'WristLeft')
    elbow_angle_right = calculate_angle(group, 'ShoulderRight', 'ElbowRight', 'WristRight')
    left_arm_angle = calculate_angle(group, 'HipLeft', 'ShoulderLeft', 'ElbowLeft')
    right_arm_angle = calculate_angle(group, 'HipRight', 'ShoulderRight', 'ElbowRight')
    arms_together_angle = calculate_angle(group, 'SpineBase', 'SpineShoulder', 'WristLeft')

    elbow_angle_left = calculate_angle(group, 'ShoulderLeft', 'ElbowLeft', 'WristLeft')
    wrist_angle_left = calculate_angle(group, 'ElbowLeft', 'WristLeft', 'HandLeft')
    shoulder_angle_left = calculate_angle(group, 'ShoulderLeft', 'SpineShoulder', 'ElbowLeft')

    elbow_angle_right = calculate_angle(group, 'ShoulderRight', 'ElbowRight', 'WristRight')
    wrist_angle_right = calculate_angle(group, 'ElbowRight', 'WristRight', 'HandRight')
    shoulder_angle_right = calculate_angle(group, 'ShoulderRight', 'SpineShoulder', 'ElbowRight')

    hip_angle = calculate_angle(group, 'HipLeft', 'SpineBase', 'KneeLeft')
    knee_angle = calculate_angle(group, 'HipLeft', 'KneeLeft', 'AnkleLeft')
    ankle_angle = calculate_angle(group, 'KneeLeft', 'AnkleLeft', 'FootLeft')

    hip_angle_right = calculate_angle(group, 'HipRight', 'SpineBase', 'KneeRight')
    knee_angle_right = calculate_angle(group, 'HipRight', 'KneeRight', 'AnkleRight')
    ankle_angle_right = calculate_angle(group, 'KneeRight', 'AnkleRight', 'FootRight')
    
    
     # Extract additional columns
    subject_id = group['SubjectID'].iloc[0]
    gesture_label = group['GestureLabel'].iloc[0]
    repetition_number = group['RepetitionNumber'].iloc[0]
    correct_label = group['CorrectLabel'].iloc[0]
    position = group['Position'].iloc[0]
    
    # Store angles and metadata in a dictionary
    angles.append({
        'SubjectID': subject_id,
        'GestureLabel': gesture_label,
        'RepetitionNumber': repetition_number,
        'CorrectLabel': correct_label,
        'Position': position,
        'ElbowAngleLeft': elbow_angle_left,
        'ElbowAngleRight': elbow_angle_right,
        'ShoulderAngleLeft': shoulder_angle_left,
        'ShoulderAngleRight': shoulder_angle_right,
        'WristAngleLeft': wrist_angle_left,
        'WristAngleRight': wrist_angle_right,
        'HipAngle': hip_angle,
        'KneeAngle': knee_angle,
        'AnkleAngle': ankle_angle,
        'HipAngleRight': hip_angle_right,
        'KneeAngleRight': knee_angle_right,
        'AnkleAngleRight': ankle_angle_right,
        'LeftArmAngle': left_arm_angle,
        'RightArmAngle': right_arm_angle,
        'ArmsTogetherAngle': arms_together_angle
    })

In [ ]:
angles_df = pd.DataFrame(angles)
angles_df

,SubjectID,GestureLabel,RepetitionNumber,CorrectLabel,Position,ElbowAngleLeft,ElbowAngleRight,LeftArmAngle,RightArmAngle,ArmsTogetherAngle
0,101,0,1,1,stand,177.953378,177.470786,23.174244,19.935830,25.940740
1,101,0,1,1,stand,177.651998,177.375867,23.299107,19.804510,25.918843
2,101,0,1,1,stand,177.894566,177.318564,23.219500,19.608224,25.930798
3,101,0,1,1,stand,177.685268,176.467396,23.247390,19.297085,25.879350
4,101,0,1,1,stand,177.856075,176.220262,23.165203,19.258214,25.859395
...,...,...,...,...,...,...,...,...,...,...
228296,307,8,9,1,stand,178.345017,175.231543,23.970470,19.467319,27.526343
228297,307,8,9,1,stand,178.787413,175.138300,24.016859,19.407770,27.534427
228298,307,8,9,1,stand,179.074571,175.183297,23.941836,19.303420,27.363165
228299,307,8,9,1,stand,179.684326,175.169224,23.944196,19.220104,27.390793


In [ ]:
angles_df.to_csv('../csvFiles/angles.csv', index=False)